In [43]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
from lxml import etree
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time
import concurrent.futures

In [44]:
col = ['Name', 'Location', 'Rank', 'Overall Score', 'Academic Reputation', 'Employer Reputation', 'Citations per Paper', 'H-index Citations', 'International Research Network']

In [45]:
ignore_international_score = ['hospitality-leisure-management', 'history', 'sociology', 'statistics-operational-research', 'law-legal-studies', 'education-training', 'business-management-studies', 'accounting-finance', 'nursing', 'modern-languages', 'english-language-literature', 'library-information-management', 'marketing', 'modern-languages', 'performing-arts', 'philosophy']

In [46]:
def QS_crawling(subject, page): 

    if page == 0:
        with pd.ExcelWriter('QS_' + subject + '_2023.xlsx', mode="w") as writer:
            output = pd.DataFrame()
            output = output = output.append(pd.DataFrame([[-1, -1, -1, -1, -1, -1, -1,-1, -1]], columns = col))
            output.to_excel(writer, index = False, sheet_name = "Sheet1")
            writer.save()
    
    output = pd.DataFrame()
    #df_existing = pd.read_excel('QS_' + subject + '_2023.xlsx')
    with pd.ExcelWriter('QS_' + subject + '_2023.xlsx', mode="a", engine = "openpyxl", if_sheet_exists='overlay') as writer:

        Rank = []
        Name = []
        Overall = []
        Academic = []
        Employer = []
        Citation = []
        H_index = []
        International = []
        Location = []
        options = Options()
        options.add_argument("--disable-notifacations") #取消彈出視窗
        options.add_argument("--window-size=5120,2880")

        browser = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

        browser.get('https://www.topuniversities.com/university-rankings/university-subject-rankings/2023/' + subject + '?&page=' + str(page) + '&tab=indicators')

        time.sleep(1)

        l = browser.find_elements(By.CLASS_NAME, 'location')
        for m in l:
            Location.append(m.text)

        element = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="block-tu-d8-content"]/div/article/div/div/div[3]/div/div[1]/div/section/div[3]/div/div/div/div[4]/span[2]')))
        
        find = browser.find_elements(By.CLASS_NAME, 'td-wrap-in')
        c = 0
        for i in find:
            if c == 0:
                c += 1
                Name.append(i.text)
            elif c == 1:
                c += 1
                Overall.append(i.text)
            elif c == 2:
                c += 1 
                Academic.append(i.text)
            elif c == 3:
                c += 1
                Employer.append(i.text)
            elif c == 4:
                c += 1
                #Citation.append(i.text)
            elif c == 5:
                c += 1
                #H_index.append(i.text)
                if subject in ignore_international_score:
                    c = 0
            elif c == 6:
                c = 0
                #International.append(i.text)
    
        for i in range(3):
            element.click()
            time.sleep(1)

        find = browser.find_elements(By.CLASS_NAME, 'td-wrap-in')
        c = 0
        for i in find:
            if c == 0:
                c += 1
                #Name.append(i.text)
            elif c == 1:
                c += 1
                #Overall.append(i.text)
            elif c == 2:
                c += 1 
                #Academic.append(i.text)
            elif c == 3:
                c += 1
                #Employer.append(i.text)
            elif c == 4:
                c += 1
                Citation.append(i.text)
            elif c == 5:
                c += 1
                H_index.append(i.text)
                if subject in ignore_international_score:
                    c = 0
            elif c == 6:
                c = 0
                International.append(i.text)

        r = browser.find_elements(By.CLASS_NAME, '_univ-rank')
        for k in r:
            Rank.append(k.text)
            
        if len(Name) == 0:
            print('end of subject')
            writer.save()
            return True
        for j in range(len(Name)):
            print(Name[j], Rank[j], Overall[j], Academic[j])
            if subject in ignore_international_score:
                output = output.append(pd.DataFrame([[Name[j], Location[j], Rank[j], Overall[j], Academic[j], Employer[j], Citation[j], H_index[j], -1]], columns = col), ignore_index=True)
            else :
                output = output.append(pd.DataFrame([[Name[j], Location[j], Rank[j], Overall[j], Academic[j], Employer[j], Citation[j], H_index[j], International[j]]], columns = col))
        browser.close()
        output.to_excel(writer, index = False, sheet_name = "Sheet1", startrow = writer.sheets['Sheet1'].max_row, header = False)
        #df_existing.to_excel(writer, index = False, sheet_name = "Sheet1")

        print('done')
        writer.save()
        return False

In [47]:
def get_subject(subject, start_page):


    while True:  
        print('crawling page', start_page)
        if QS_crawling(subject, start_page):
            break
        start_page += 1

In [48]:
subjects = ['education-training', 'business-management-studies', 'accounting-finance']

get_subject('philosophy', 0)

crawling page 0


C:\Users\MSI\AppData\Local\Temp\ipykernel_26616\4148808711.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output = output.append(pd.DataFrame([[-1, -1, -1, -1, -1, -1, -1,-1, -1]], columns = col))
C:\Users\MSI\AppData\Local\Temp\ipykernel_26616\4148808711.py:8: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
D:\system\program\Anaconda\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
C:\Users\MSI\AppData\Local\Temp\ipykernel_26616\4148808711.py:27: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
C:\Users\MSI\AppData\Local\Temp\ipykernel_26616\4148808711.py:27: Deprecation

New York University (NYU) 1 97.6 99.6
Rutgers University–New Brunswick 2 96.8 100
The London School of Economics and Political Science (LSE) 3 94.3 96
University of Oxford 4 92 91.2
University of Pittsburgh 5 90.6 95.5
University of Cambridge 6 89.6 89.4
Harvard University 7 88.3 88.1
Australian National University (ANU) 8 88.1 88.2
Ludwig-Maximilians-Universität München 9 87.8 89.7
University of St Andrews 10 87.5 89.5
University of Toronto 11 86.4 87.1
University of Notre Dame 12 86.3 88.3
Humboldt-Universität zu Berlin 13 86.1 87.6
Princeton University 14 85.5 84.2
Yale University 15 84.2 82.9
done


C:\Users\MSI\AppData\Local\Temp\ipykernel_26616\4148808711.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(pd.DataFrame([[Name[j], Location[j], Rank[j], Overall[j], Academic[j], Employer[j], Citation[j], H_index[j], -1]], columns = col), ignore_index=True)
C:\Users\MSI\AppData\Local\Temp\ipykernel_26616\4148808711.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(pd.DataFrame([[Name[j], Location[j], Rank[j], Overall[j], Academic[j], Employer[j], Citation[j], H_index[j], -1]], columns = col), ignore_index=True)
C:\Users\MSI\AppData\Local\Temp\ipykernel_26616\4148808711.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(pd.DataFrame([[Name[j], L

crawling page 1


...

C:\Users\MSI\AppData\Local\Temp\ipykernel_26616\4148808711.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(pd.DataFrame([[Name[j], Location[j], Rank[j], Overall[j], Academic[j], Employer[j], Citation[j], H_index[j], -1]], columns = col), ignore_index=True)
C:\Users\MSI\AppData\Local\Temp\ipykernel_26616\4148808711.py:108: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(pd.DataFrame([[Name[j], Location[j], Rank[j], Overall[j], Academic[j], Employer[j], Citation[j], H_index[j], -1]], columns = col), ignore_index=True)
C:\Users\MSI\AppData\Local\Temp\ipykernel_26616\4148808711.py:116: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


crawling page 15


C:\Users\MSI\AppData\Local\Temp\ipykernel_26616\4148808711.py:27: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
C:\Users\MSI\AppData\Local\Temp\ipykernel_26616\4148808711.py:27: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)


end of subject


C:\Users\MSI\AppData\Local\Temp\ipykernel_26616\4148808711.py:103: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
